Here is just a test notebook.

In [55]:
#import packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u  
import time

I have two files, one from Einstein Observatory and one from ROSAT observatory. They both have detections. Each row in both files are a source detection. I want to see if they could be connected. My first way is to see if they are nearby each other. I want to compare every line in Einstein with every line in ROSAT. I'm trying to do this as fast as possible, or at least a way that isn't taking over 5 days.... these for loops are slow. I tried your np.where suggestion but I still have a for loop and I was wondering if there is a better way to do this.

In [57]:
#Here is one example
#here is all the data. The Einstein file has 46,000 lines and the ROSAT has 112,000 lines
#this takes almost 3 minutes

start = time.time() #start a time to see how long it takes

ROSAT = pd.read_csv("2rxs_lite.txt", delim_whitespace=True)  
Einstein = pd.read_csv("2sig_J2000_theta.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio

count = 0

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?

    tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(180/np.pi)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp) ==0: #is there a better way to do this? instead of the if... else statements
            continue

    else: 
        count += 1 #just to see how many matches we get
print(count)


print(f'Numpy: {time.time() - start} seconds')


46185
Numpy: 155.67349290847778 seconds


In [128]:
start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("ROSAT-sorted.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = [] #the angular separation

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?

    tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_Einstein[i]*(180/np.pi)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp) ==0: 
            continue
    else: 
        for j in tmp:
                count+= 1
                theta_rad = np.sin(DEC_Einstein[i]*180/np.pi)* np.sin(DEC_ROSAT[j]*180/np.pi) + np.cos(DEC_Einstein[i]*180/np.pi)*np.cos(DEC_ROSAT[j]*180/np.pi)*np.cos(np.absolute(RA_Einstein[i]-RA_ROSAT[i])*180/np.pi) #returns answer in rad
                theta_arcsec = np.arccos(theta_rad) * 206265
                theta.append(theta_arcsec) 
print(count)

print(np.size(theta))
print(f'Numpy: {time.time() - start} seconds')


328
328
Numpy: 1.5206270217895508 seconds


/Users/snegussie/Library/Python/3.8/lib/python/site-packages/numpy/core/fromnumeric.py:3245: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


However, I want to do more than just find out how many lines have coordinates that could be near each other so I want to save those coordinates. I took the previous code and am adding to it.

In [188]:
start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("rass_short.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = np.zeros(638) #the angular separation
k = 0

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?
        #print("i =", i)
   
        tmp = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(np.pi/180)) )  < 1 ) )  #conditions it need to meet
        print(type(tmp))
        if np.size(tmp) ==0: 
                continue
        else: 
                for j in tmp:
                        count+= 1
                        theta_rad = np.sin(DEC_Einstein[i]*np.pi/180)* np.sin(DEC_ROSAT[j]*np.pi/180) + np.cos(DEC_Einstein[i]*np.pi/180)*np.cos(DEC_ROSAT[j]*np.pi/180)*np.cos(np.absolute(RA_Einstein[i]-RA_ROSAT[i])*np.pi/180) #returns answer in rad
                        theta_arcsec = np.arccos(theta_rad) * 206265
                        print(theta_arcsec)
                        print(type(theta_arcsec))
                        #print(np.size(theta_arcsec))
                        #L = [k if theta_arcsec < 300]
                        #if theta_arcsec < 300:
                        if (theta_arcsec < 300).all: 
                                np.append(theta, theta_arcsec)
                                tmp_num = theta_arcsec
                                k += 1
                        #print(theta)
                
print(count)
print(np.size(theta))
#a = np.sin(90*np.pi/180)
#print(a)

ang_sep = np.zeros(count)
#for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
#    ang_sep[i] = theta[0][i]

#print(np.shape(tmp))
print("Writing file")

with open(r'test.txt', 'w') as fp:
        for i in range(count):
            if i == 0:
                fp.write("Theta\n")
            else:
                fp.write("%f\n" %(theta[i]))

print(f'Numpy: {time.time() - start} seconds')


<class 'tuple'>
0      138.814945
1      138.814945
2      588.359061
3      787.241455
4      975.382096
5     1243.443356
6     1369.743136
7     1338.472574
8     1525.524728
9     1629.088507
10    1763.683722
11    1854.889768
12    1882.840956
13    2076.673184
14    2056.870073
15    2141.082433
16    2076.673184
17    2141.082433
18    2379.035200
19    2235.478571
20    2247.233979
21    2340.887758
22    2382.091378
23    2494.169676
24    2628.520467
25    2835.000189
26    3045.571491
27    3109.351808
28    3280.053332
29    3464.078717
Name: 55.12909, dtype: float64
<class 'pandas.core.series.Series'>
<class 'tuple'>
0       25.811705
1       25.811705
2      480.722026
3      682.691742
4      872.578210
5     1142.206827
6     1269.031183
7     1237.640013
8     1425.339613
9     1529.197661
10    1664.123508
11    1755.526248
12    1783.533890
13    1977.715704
14    1957.879906
15    2042.227078
16    1977.715704
17    2042.227078
18    2280.509101
19    2136.762235
2

In [ ]:
L = [i for i in range (1, 10000000) if i%130 == 0]
tmp = [i for i in range (0, np.size(DEC_Einstein)) if np.where( ((np.absolute(tmp_num - DEC_ROSAT) < 1)) & (np.absolute(RA_Einstein-RA_ROSAT) * np.cos(DEC_ROSAT*(180/np.pi))  < 1 ) ) ]
np.size(tmp)

638

In [60]:
start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("ROSAT-sorted.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = [] #the angular separation
offset = [] # error in measurements

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?
    tmp_array = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & 
                   ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_Einstein[i]*(np.pi/180)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp_array) ==0: 
            continue
    else: 

        #for j in range(np.size(tmp_array)):
        tmp_num = tmp_array[0][0]
        count+= 1
        theta_rad = np.sin(DEC_Einstein[tmp_num]*np.pi/180)* np.sin(DEC_ROSAT[tmp_num]*np.pi/180) + np.cos(DEC_Einstein[tmp_num]*np.pi/180)*np.cos(DEC_ROSAT[tmp_num]*np.pi/180)*np.cos(np.absolute(RA_Einstein[tmp_num]-RA_ROSAT[tmp_num])*np.pi/180) #returns answer in rad
        theta_arcsec = np.arccos(theta_rad) * 206265
        #print(theta_arcsec)
        
        #SNR
        if theta_arcsec <= 300: 
            if SNR_Einstein[i] >= 2 and SNR_Einstein[i] < 3:
                    poserr1 = 73 
            elif SNR_Einstein[i] >= 3 and SNR_Einstein[i] < 4:
                    poserr1 = 53 
            elif SNR_Einstein[i] >= 4 and SNR_Einstein[i] < 6:
                    poserr1 = 41 
            elif SNR_Einstein[i] >= 6:
                    poserr1 = 30 
            else:
                    continue
                    
            poserr2 = SNR_ROSAT[j]
            confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
            offset.append(confid)
            theta.append(theta_arcsec) 

#print(np.size(theta))
#print(np.size(count))
"""
ang_sep = np.zeros(count)
for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
    ang_sep[i] = float("{:.1f}".format(theta[i]))

confidence_new = np.zeros(count)
for i in range(count):
    #ra_E_new[i] = round(RA_E[i].value, 1)
    confidence_new[i] = float("{:.2f}".format(offset[i]))

print("Writing file")

with open(r'test.txt', 'w') as fp:
        for i in range(count):
            if i == 0:
                fp.write("\tTheta\t\tOffset\n")
            else:
                fp.write("%8.1f\t%8.2f\n" %((ang_sep[i]), (confidence_new[i])  ))
"""
print(np.size(theta))
print("finished")

print(f'Numpy: {time.time() - start} seconds')

In [123]:
start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("rass_.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = [] #the angular separation
offset = [] # error in measurements

for i in range(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?
    tmp_array = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & 
                   ( (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_Einstein[i]*(np.pi/180)) )  < 1 ) )  #conditions it need to meet

    if np.size(tmp_array) ==0: 
            continue
    else: 

        #for j in range(np.size(tmp_array)):
        tmp_num = tmp_array[0][0]
        count+= 1
        theta_rad = np.sin(DEC_Einstein[tmp_num]*np.pi/180)* np.sin(DEC_ROSAT[tmp_num]*np.pi/180) + np.cos(DEC_Einstein[tmp_num]*np.pi/180)*np.cos(DEC_ROSAT[tmp_num]*np.pi/180)*np.cos(np.absolute(RA_Einstein[tmp_num]-RA_ROSAT[tmp_num])*np.pi/180) #returns answer in rad
        theta_arcsec = np.arccos(theta_rad) * 206265
        #print(theta_arcsec)
        
        #SNR
        if theta_arcsec <= 300: 
            if SNR_Einstein[i] >= 2 and SNR_Einstein[i] < 3:
                    poserr1 = 73 
            elif SNR_Einstein[i] >= 3 and SNR_Einstein[i] < 4:
                    poserr1 = 53 
            elif SNR_Einstein[i] >= 4 and SNR_Einstein[i] < 6:
                    poserr1 = 41 
            elif SNR_Einstein[i] >= 6:
                    poserr1 = 30 
            else:
                    continue
                    
            poserr2 = SNR_ROSAT[j]
            confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
            offset.append(confid)
            theta.append(theta_arcsec) 

#print(np.size(theta))
#print(np.size(count))
"""
ang_sep = np.zeros(count)
for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
    ang_sep[i] = float("{:.1f}".format(theta[i]))

confidence_new = np.zeros(count)
for i in range(count):
    #ra_E_new[i] = round(RA_E[i].value, 1)
    confidence_new[i] = float("{:.2f}".format(offset[i]))

print("Writing file")

with open(r'test.txt', 'w') as fp:
        for i in range(count):
            if i == 0:
                fp.write("\tTheta\t\tOffset\n")
            else:
                fp.write("%8.1f\t%8.2f\n" %((ang_sep[i]), (confidence_new[i])  ))
"""
print(np.size(theta))
print("finished")

print(f'Numpy: {time.time() - start} seconds')

KeyError: 2457

In [110]:
#tesing


#works now


start = time.time() #start a time to see how long it takes

#Here is the other example with shorter files
#this one only takes 0.7 seconds

ROSAT = pd.read_csv("rass_short.txt", delim_whitespace=True)  
Einstein = pd.read_csv("ets_short.txt", delim_whitespace=True)

RA_ROSAT = ROSAT.iloc[:, 2] #right ascention
DEC_ROSAT = ROSAT.iloc[:, 3] #decllination
SNR_ROSAT = ROSAT.iloc[:, 4] #signal to noise ratio

RA_Einstein = Einstein.iloc[:, 0] #right ascention
DEC_Einstein = Einstein.iloc[:, 1] #decllination
SNR_Einstein = Einstein.iloc[:, 2] #signal to noise ratio
count = 0

theta = [] #the angular separation
offset = [] # error in measurements

for i in range(10):#(np.size(DEC_Einstein)): # i was wondering, is there a better way to do this instead of a for loop since it is so slow?
    print("i =", i)
    tmp_array = np.where( ((np.absolute(DEC_Einstein[i] - DEC_ROSAT) < 1)) & (np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_Einstein[i]*(np.pi/180))   < 1) )  #conditions it need to meet

    #print(np.absolute(RA_Einstein[i]-RA_ROSAT[i]) * np.cos(DEC_ROSAT*(np.pi/180)))
    print(tmp_array)
    print("tmp arrary size= ",np.size(tmp_array))
#    print("t2",np.size(tmp_array2))
    #print(DEC_ROSAT)
    if np.size(tmp_array) ==0: 
            continue
    else: 
        count += 1

        for j in range(np.size(tmp_array)):
                count += 1
                tmp_num = tmp_array[0][0]
                theta_rad = np.sin(DEC_Einstein[tmp_num]*np.pi/180)* np.sin(DEC_ROSAT[tmp_num]*np.pi/180) + np.cos(DEC_Einstein[tmp_num]*np.pi/180)*np.cos(DEC_ROSAT[tmp_num]*np.pi/180)*np.cos(np.absolute(RA_Einstein[tmp_num]-RA_ROSAT[tmp_num])*np.pi/180) #returns answer in rad
                theta_arcsec = np.arccos(theta_rad) * 206265
                #print(theta_arcsec)
                
                #SNR
                if theta_arcsec <= 300: 
                        

                        if SNR_Einstein[i] >= 2 and SNR_Einstein[i] < 3:
                                poserr1 = 73 
                        elif SNR_Einstein[i] >= 3 and SNR_Einstein[i] < 4:
                                poserr1 = 53 
                        elif SNR_Einstein[i] >= 4 and SNR_Einstein[i] < 6:
                                poserr1 = 41 
                        elif SNR_Einstein[i] >= 6:
                                poserr1 = 30 
                        else:
                                continue
                                
                        poserr2 = SNR_ROSAT[i]
                        confid = theta_arcsec/np.sqrt(poserr1**2 + poserr2**2) #finding theta hat
                        
                        offset.append(confid)
                        theta.append(theta_arcsec) 

#print(np.size(theta))
#print(np.size(count))
"""
ang_sep = np.zeros(count)
for i in range(count):
    #ang_sep[i] = round(theta[i].value, 3)
    ang_sep[i] = float("{:.1f}".format(theta[i]))

confidence_new = np.zeros(count)
for i in range(count):
    #ra_E_new[i] = round(RA_E[i].value, 1)
    confidence_new[i] = float("{:.2f}".format(offset[i]))

print("Writing file")

with open(r'test.txt', 'w') as fp:
        for i in range(count):
            if i == 0:
                fp.write("\tTheta\tOffset\n")
            else:
                fp.write("%8.1f\t%8.2f\n" %((ang_sep[i]), (confidence_new[i])  ))
"""
print(np.size(count))
print("finished")

print(f'Numpy: {time.time() - start} seconds')

i = 0
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),)
tmp arrary size=  30
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
i = 1
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),)
tmp arrary size=  31
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
i = 2
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),)
tmp arrary size=  31
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
i = 3
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 

In [189]:
ROSAT = np.loadtxt("rass_short.txt", delimiter=" ")  
Einstein = np.loadtxt("ets_short.txt", delimiter=" ")

RA_ROSAT = ROSAT[:, 2] #right ascention
DEC_ROSAT = ROSAT[:, 3] #decllination
SNR_ROSAT = ROSAT[:, 4] #signal to noise ratio

RA_Einstein = Einstein[:, 0] #right ascention
DEC_Einstein = Einstein[:, 1] #decllination
SNR_Einstein = Einstein[:, 2] #signal to noise ratio

ValueError: could not convert string to float: '2RXS'